# Splunk SIEM Server for Phishing Detection

This notebook runs a **real Splunk instance** in Google Colab using `udocker` (user-space Docker).

## Architecture
```
┌─────────────────────────┐         ┌─────────────────────────┐
│   COLAB NOTEBOOK 1      │         │   COLAB NOTEBOOK 2      │
│   (This Notebook)       │         │   (Phishing Detection)  │
│                         │         │                         │
│   Splunk via udocker    │◄────────│   FastAPI Gateway       │
│   - Web UI (8000)       │   HEC   │   - RF, XGBoost, LLM    │
│   - HEC (8088)          │         │                         │
│                         │         │                         │
│   Exposed via ngrok     │         │                         │
└─────────────────────────┘         └─────────────────────────┘
```

## What is udocker?
- Runs Docker containers **without root privileges**
- Works in Google Colab (unlike regular Docker)
- Uses PRoot for user-space container execution

---

## 1. Install udocker & Dependencies

In [ ]:
# @title 1.1 Install udocker and ngrok
# @markdown This installs udocker (user-space Docker) and ngrok for tunneling

import os
import sys

print("Installing udocker...")
!pip install -q udocker pyngrok requests

# Initialize udocker
print("\nInitializing udocker...")
!udocker install

print("\n✓ udocker installed successfully!")
!udocker version

In [ ]:
# @title 1.2 Configure ngrok (REQUIRED)
# @markdown Get your free ngrok token from: https://dashboard.ngrok.com/get-started/your-authtoken

NGROK_AUTH_TOKEN = ""  # @param {type:"string"}

if NGROK_AUTH_TOKEN:
    from pyngrok import ngrok
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✓ ngrok authenticated")
else:
    raise ValueError(
        "\n" + "="*60 + "\n" +
        "ERROR: ngrok token is REQUIRED!\n" +
        "="*60 + "\n" +
        "1. Go to: https://dashboard.ngrok.com/signup\n" +
        "2. Sign up for free and verify your email\n" +
        "3. Go to: https://dashboard.ngrok.com/get-started/your-authtoken\n" +
        "4. Copy your authtoken and paste it above\n" +
        "="*60
    )

## 2. Configure Splunk Settings

In [ ]:
# @title 2.1 Splunk Configuration
# @markdown Set your Splunk admin password and HEC token

SPLUNK_PASSWORD = "PhishingDemo123!"  # @param {type:"string"}
SPLUNK_HEC_TOKEN = "phishing-hec-token-demo"  # @param {type:"string"}
SPLUNK_INDEX = "security"  # @param {type:"string"}

# Validate
if len(SPLUNK_PASSWORD) < 8:
    raise ValueError("Password must be at least 8 characters!")

print(f"✓ Splunk admin password set (min 8 chars)")
print(f"✓ HEC Token: {SPLUNK_HEC_TOKEN}")
print(f"✓ Index: {SPLUNK_INDEX}")

## 3. Pull and Run Splunk Container

In [ ]:
# @title 3.1 Pull Splunk Image
# @markdown This downloads the Splunk container image (may take 5-10 minutes)

print("Pulling Splunk image... (this takes 5-10 minutes)")
print("Image size is ~1.5GB\n")

!udocker pull splunk/splunk:latest

print("\n✓ Splunk image pulled successfully!")
!udocker images

In [ ]:
# @title 3.2 Create Splunk Container
# @markdown Creates a container from the pulled image

import subprocess

# Remove existing container if any
print("Removing any existing splunk container...")
!udocker rm splunk 2>/dev/null || true

# Create new container
print("\nCreating Splunk container...")
!udocker create --name=splunk splunk/splunk:latest

print("\n✓ Container created!")
!udocker ps

In [ ]:
# @title 3.3 Configure Container Execution Mode
# @markdown Sets up PRoot execution mode for the container

# Use P2 mode (PRoot without seccomp) which works better in Colab
print("Setting execution mode to P2 (PRoot)...")
!udocker setup --execmode=P2 splunk

print("\n✓ Execution mode configured")

In [ ]:
# @title 3.4 Start Splunk Container
# @markdown Starts Splunk in the background

import subprocess
import threading
import time
import os

# Environment variables for Splunk
splunk_env = {
    'SPLUNK_START_ARGS': '--accept-license',
    'SPLUNK_PASSWORD': SPLUNK_PASSWORD,
    'SPLUNK_HEC_TOKEN': SPLUNK_HEC_TOKEN,
}

env_args = ' '.join([f'-e {k}={v}' for k, v in splunk_env.items()])

print("Starting Splunk container...")
print("This will take 2-3 minutes for Splunk to initialize.\n")

# Start Splunk in background
def run_splunk():
    cmd = f'udocker run {env_args} -p 8000:8000 -p 8088:8088 -p 8089:8089 splunk /sbin/entrypoint.sh start-service'
    os.system(cmd)

splunk_thread = threading.Thread(target=run_splunk, daemon=True)
splunk_thread.start()

print("Splunk starting in background...")
print("Waiting for Splunk to be ready (checking every 10 seconds)...\n")

In [ ]:
# @title 3.5 Wait for Splunk to be Ready
# @markdown Checks if Splunk is responding

import requests
import time

print("Waiting for Splunk to start...")
print("(This typically takes 2-3 minutes)\n")

max_wait = 300  # 5 minutes max
start_time = time.time()
ready = False

while time.time() - start_time < max_wait:
    try:
        r = requests.get("http://localhost:8000", timeout=5)
        if r.status_code in [200, 303]:
            ready = True
            print("\n✓ Splunk Web UI is ready!")
            break
    except:
        pass
    
    elapsed = int(time.time() - start_time)
    print(f"  Still starting... ({elapsed}s)", end="\r")
    time.sleep(10)

if not ready:
    print("\n⚠ Splunk is taking longer than expected.")
    print("You can continue anyway - it may still be starting.")

## 4. Configure HEC (HTTP Event Collector)

In [ ]:
# @title 4.1 Configure Splunk HEC
# @markdown Creates the index and HEC token

import requests
import time
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

SPLUNK_API = "https://localhost:8089"
auth = ("admin", SPLUNK_PASSWORD)

print("Configuring Splunk HEC...\n")

# Wait for management port
print("Waiting for Splunk management API...")
for i in range(30):
    try:
        r = requests.get(f"{SPLUNK_API}/services", auth=auth, verify=False, timeout=5)
        if r.status_code in [200, 401]:
            print("  ✓ Management API responding")
            break
    except:
        time.sleep(5)

# Step 1: Create the security index
print(f"\n1. Creating index '{SPLUNK_INDEX}'...")
try:
    r = requests.post(
        f"{SPLUNK_API}/services/data/indexes",
        auth=auth,
        verify=False,
        data={"name": SPLUNK_INDEX}
    )
    if r.status_code in [201, 409]:
        print(f"   ✓ Index '{SPLUNK_INDEX}' ready")
    else:
        print(f"   ⚠ Index creation returned: {r.status_code}")
except Exception as e:
    print(f"   ⚠ Error: {e}")

# Step 2: Enable HEC globally
print("\n2. Enabling HEC globally...")
try:
    r = requests.post(
        f"{SPLUNK_API}/services/data/inputs/http/http",
        auth=auth,
        verify=False,
        data={"disabled": "0", "enableSSL": "0"}  # Disable SSL for simplicity
    )
    print("   ✓ HEC enabled")
except Exception as e:
    print(f"   Note: {e}")

# Step 3: Create HEC token
print(f"\n3. Creating HEC token...")
try:
    # Delete existing token if any
    requests.delete(
        f"{SPLUNK_API}/services/data/inputs/http/phishing_detection",
        auth=auth,
        verify=False
    )
    
    # Create new token
    r = requests.post(
        f"{SPLUNK_API}/services/data/inputs/http",
        auth=auth,
        verify=False,
        data={
            "name": "phishing_detection",
            "token": SPLUNK_HEC_TOKEN,
            "index": SPLUNK_INDEX,
            "indexes": SPLUNK_INDEX,
            "sourcetype": "phishing_detection",
            "disabled": "0",
            "useACK": "0"
        }
    )
    if r.status_code in [200, 201, 409]:
        print(f"   ✓ HEC token created: {SPLUNK_HEC_TOKEN}")
    else:
        print(f"   Status: {r.status_code}")
except Exception as e:
    print(f"   ⚠ Error: {e}")

print("\n✓ Splunk HEC configuration complete!")

## 5. Create ngrok Tunnel

In [ ]:
# @title 5.1 Create Public URL for HEC
# @markdown Creates an ngrok tunnel so other notebooks can reach Splunk

from pyngrok import ngrok
import time

# Kill any existing tunnels
ngrok.kill()
time.sleep(2)

# Create tunnel for HEC (port 8088)
print("Creating ngrok tunnel for Splunk HEC...")
hec_tunnel = ngrok.connect(8088, "http")
HEC_PUBLIC_URL = hec_tunnel.public_url

# Convert to HTTPS
if HEC_PUBLIC_URL.startswith("http://"):
    HEC_PUBLIC_URL = HEC_PUBLIC_URL.replace("http://", "https://")

SPLUNK_HEC_FULL_URL = f"{HEC_PUBLIC_URL}/services/collector"

print("\n" + "="*70)
print("🎉 SPLUNK IS READY!")
print("="*70)
print(f"\n📊 Splunk Web UI (local): http://localhost:8000")
print(f"   Username: admin")
print(f"   Password: {SPLUNK_PASSWORD}")
print(f"\n🔗 HEC Endpoint (PUBLIC - use this in phishing notebook):")
print(f"   {SPLUNK_HEC_FULL_URL}")
print(f"\n🔑 HEC Token: {SPLUNK_HEC_TOKEN}")
print(f"📁 Index: {SPLUNK_INDEX}")
print("="*70)

In [ ]:
# @title 5.2 Copy These Settings
# @markdown Run this cell to get the configuration for your phishing notebook

print("="*70)
print("📋 COPY THESE SETTINGS TO YOUR PHISHING DETECTION NOTEBOOK")
print("="*70)
print(f'''
# Splunk Configuration
SPLUNK_HEC_URL = "{SPLUNK_HEC_FULL_URL}"
SPLUNK_HEC_TOKEN = "{SPLUNK_HEC_TOKEN}"
SPLUNK_INDEX = "{SPLUNK_INDEX}"
''')
print("="*70)
print("\nOr configure via API after starting the phishing server:")
print(f'''
import requests
requests.post(
    f"{{API_URL}}/splunk/configure",
    params={{
        "hec_url": "{SPLUNK_HEC_FULL_URL}",
        "token": "{SPLUNK_HEC_TOKEN}",
        "index": "{SPLUNK_INDEX}"
    }}
)
''')

## 6. Test HEC Connection

In [ ]:
# @title 6.1 Test HEC Connection
# @markdown Sends a test event to verify HEC is working

import requests
import json
from datetime import datetime
import urllib3
urllib3.disable_warnings()

print("Testing HEC connection...\n")

test_event = {
    "time": datetime.now().timestamp(),
    "host": "colab-test",
    "source": "phishing-gateway-test",
    "sourcetype": "phishing_detection",
    "index": SPLUNK_INDEX,
    "event": {
        "alert_id": f"test-{datetime.now().strftime('%H%M%S')}",
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "event_type": "connection_test",
        "severity": "INFO",
        "message": "HEC connection test from Colab",
        "detection": {
            "classification": "TEST",
            "is_phishing": False,
            "probability": 0.0,
            "risk_score": 0
        }
    }
}

headers = {
    "Authorization": f"Splunk {SPLUNK_HEC_TOKEN}",
    "Content-Type": "application/json"
}

# Test local first
print("Testing local HEC (localhost:8088)...")
try:
    r = requests.post(
        "http://localhost:8088/services/collector",
        headers=headers,
        json=test_event,
        timeout=10
    )
    if r.status_code == 200:
        print(f"   ✓ Local HEC working! Response: {r.json()}")
    else:
        print(f"   ⚠ Status {r.status_code}: {r.text}")
except Exception as e:
    print(f"   ⚠ Local test failed: {e}")

# Test via ngrok
print(f"\nTesting public HEC ({HEC_PUBLIC_URL})...")
try:
    r = requests.post(
        SPLUNK_HEC_FULL_URL,
        headers=headers,
        json=test_event,
        timeout=10
    )
    if r.status_code == 200:
        print(f"   ✓ Public HEC working! Response: {r.json()}")
        print("\n" + "="*50)
        print("✅ READY TO RECEIVE PHISHING ALERTS!")
        print("="*50)
    else:
        print(f"   ⚠ Status {r.status_code}: {r.text}")
except Exception as e:
    print(f"   ⚠ Public test failed: {e}")

## 7. Monitor Events

In [ ]:
# @title 7.1 Search for Events in Splunk
# @markdown Queries Splunk for received phishing detection events

import requests
import json
import urllib3
urllib3.disable_warnings()

print(f"Searching for events in index='{SPLUNK_INDEX}'...\n")

search_query = f"search index={SPLUNK_INDEX} | head 20"

try:
    r = requests.post(
        "https://localhost:8089/services/search/jobs",
        auth=("admin", SPLUNK_PASSWORD),
        verify=False,
        data={
            "search": search_query,
            "output_mode": "json",
            "exec_mode": "oneshot"
        },
        timeout=30
    )
    
    if r.status_code == 200:
        results = r.json()
        if "results" in results and results["results"]:
            print(f"Found {len(results['results'])} events:\n")
            for i, event in enumerate(results["results"], 1):
                print(f"{'='*60}")
                print(f"Event {i}")
                print(f"{'='*60}")
                raw = event.get("_raw", str(event))
                try:
                    parsed = json.loads(raw)
                    print(json.dumps(parsed, indent=2)[:800])
                except:
                    print(raw[:800])
                print()
        else:
            print("No events found yet.")
            print("Send some phishing predictions from the main notebook!")
    else:
        print(f"Search returned status {r.status_code}")
except Exception as e:
    print(f"Error searching: {e}")

In [ ]:
# @title 7.2 Keep Alive & Monitor
# @markdown Keeps the notebook running and shows event count

import time
import requests
import urllib3
urllib3.disable_warnings()

print("🔄 Keep-alive running. Splunk is accepting events.")
print(f"   HEC URL: {SPLUNK_HEC_FULL_URL}")
print(f"   Token: {SPLUNK_HEC_TOKEN}")
print("\nPress STOP (⬛) to end.\n")
print("="*50)

event_count = 0
start_time = time.time()

while True:
    try:
        # Check event count
        try:
            search_r = requests.post(
                "https://localhost:8089/services/search/jobs",
                auth=("admin", SPLUNK_PASSWORD),
                verify=False,
                data={
                    "search": f"search index={SPLUNK_INDEX} | stats count",
                    "output_mode": "json",
                    "exec_mode": "oneshot"
                },
                timeout=10
            )
            if search_r.status_code == 200:
                results = search_r.json().get("results", [])
                if results:
                    event_count = int(results[0].get("count", 0))
        except:
            pass
        
        elapsed = int(time.time() - start_time)
        hours, remainder = divmod(elapsed, 3600)
        minutes, seconds = divmod(remainder, 60)
        
        print(f"\r[{hours:02d}:{minutes:02d}:{seconds:02d}] Splunk: ✓ Running | Events indexed: {event_count}", end="")
        
    except Exception as e:
        print(f"\r⚠ Health check error: {e}", end="")
    
    time.sleep(30)

---

## Alternative: Mock HEC Server (If udocker doesn't work)

If udocker has issues, you can use a simple Python server that mimics Splunk HEC:

In [ ]:
# @title Alternative: Start Mock HEC Server
# @markdown Use this if the udocker Splunk doesn't work

from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading
import json
from datetime import datetime
import time

!pip install -q flask

MOCK_HEC_TOKEN = "mock-hec-token-12345"  # @param {type:"string"}
PORT = 8088

events_log = []
mock_app = Flask(__name__)

@mock_app.route('/services/collector', methods=['POST'])
@mock_app.route('/services/collector/event', methods=['POST'])
def receive_event():
    auth_header = request.headers.get('Authorization', '')
    if auth_header != f"Splunk {MOCK_HEC_TOKEN}":
        return jsonify({"text": "Invalid token", "code": 4}), 403
    
    try:
        data = request.get_json()
        events_log.append({"received_at": datetime.now().isoformat(), "data": data})
        severity = data.get('event', {}).get('severity', 'INFO')
        alert_id = data.get('event', {}).get('alert_id', 'unknown')
        print(f"\n📨 EVENT [{severity}] - {alert_id}")
        print(json.dumps(data.get('event', {}), indent=2)[:500])
        return jsonify({"text": "Success", "code": 0}), 200
    except Exception as e:
        return jsonify({"text": str(e), "code": 6}), 400

@mock_app.route('/events', methods=['GET'])
def get_events():
    return jsonify({"total": len(events_log), "events": events_log[-50:]})

def run_mock():
    mock_app.run(host='0.0.0.0', port=PORT, debug=False, use_reloader=False)

threading.Thread(target=run_mock, daemon=True).start()
time.sleep(2)

ngrok.kill()
time.sleep(1)
tunnel = ngrok.connect(PORT, "http")
MOCK_URL = tunnel.public_url.replace("http://", "https://")

print("\n" + "="*70)
print("🎉 MOCK SPLUNK HEC IS RUNNING!")
print("="*70)
print(f'\nSPLUNK_HEC_URL = "{MOCK_URL}/services/collector"')
print(f'SPLUNK_HEC_TOKEN = "{MOCK_HEC_TOKEN}"')
print(f'SPLUNK_INDEX = "security"')
print("="*70)